In [2]:
!pip install surprise

You should consider upgrading via the 'C:\Users\sapb2\anaconda3\python.exe -m pip install --upgrade pip' command.



  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp38-cp38-win_amd64.whl size=718085 sha256=10ea3e71254029b964032e2a9df5f0b9b8458f8648d33d051ce410b8b2bfbb74
  Stored in directory: c:\users\sapb2\appdata\local\pip\cache\wheels\20\91\57\2965d4cff1b8ac7ed1b6fa25741882af3974b54a31759e10b6
Successfully built scikit-surprise


In [3]:
import surprise
print(surprise.__version__)

1.1.1


## Surprise 를 이용한 추천 시스템 구축

In [4]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

<h4> 내장 데이터를 로드하고 학습과 테스트 데이터로 분리

In [5]:
data = Dataset.load_builtin("ml-100k")
trainset, testset = train_test_split(data, test_size= .25, random_state = 0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\sapb2/.surprise_data/ml-100k


<h4> 추천 행렬 분해 알고리즘으로 SVD객체를 생성하고 학습수행

In [6]:
algo = SVD()
algo.fit(trainset)

<h4> 테스트 데이터 세트에 예상 평점 데이터 예측. </h4><p>test()메소드 호출 시 Prediction 객체의 리스트로 평점 예측 데이터 반환

In [7]:
predictions = algo.test(testset)
print("prediction type : " ,type(predictions), "size : ",len(predictions))
print("rediction 결과의 최초 5개 추출 ")
predictions[:5]

prediction type :  <class 'list'> size :  25000
rediction 결과의 최초 5개 추출 


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.525880042609959, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.805243490762366, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=3.9488593045979483, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.730397725738154, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.5267865043058952, details={'was_impossible': False})]

In [11]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:5]]

[('120', '282', 3.525880042609959),
 ('882', '291', 3.805243490762366),
 ('535', '507', 3.9488593045979483),
 ('697', '244', 3.730397725738154),
 ('751', '385', 3.5267865043058952)]

<h4> predict()메소드는 개별 사용자,아이템에 대한 예측 평점 정보를 반환

In [9]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함. 
uid = str(196)
iid = str(302)
pred = algo.predict(uid,iid)
# test() 메소드와 달리 predict는 1개만 예측
print(pred)

user: 196        item: 302        r_ui = None   est = 4.31   {'was_impossible': False}


<h4> 반환된 Prediction의 리스트 객체를 기반으로 RMSE 평가

In [10]:
accuracy.rmse(predictions)

RMSE: 0.9451


0.945120602431736

## Surprise 주요 모듈 소개
<h4>csv 파일로 사용자 평점 데이터 생성

In [12]:
import pandas as pd

ratings = pd.read_csv("./ml-latest-small/ratings.csv")
# ratings_noh.csv 파일로 unload 시 index 와 header를 모두 제거한 새로운 파일 생성.  
ratings.to_csv('./ml-latest-small/ratings_noh.csv', index=False, header=False)

<h4> Reader클래스로 파일의 포맷팅 지정하고 Dataset의 load_from_file()을 이용하여 데이터셋 로딩

In [13]:
from surprise import Reader

# csv에서 긁어와서 sep = ',' 로 reader 해줘야함
reader = Reader(line_format = 'user item rating timestamp', sep=',', rating_scale = (0.5, 5))
data = Dataset.load_from_file('./ml-latest-small/ratings_noh.csv',reader = reader)

<h4> 학습과 테스트 데이터 세트로 분할하고 SVD로 학습후 테스트데이터 평점 예측 후 RMSE평가

In [15]:
trainset, testset = train_test_split(data, test_size= .25, random_state = 0)

# 수행시마다 동일한 결과 도출을 위해 random_state 설정 
algo = SVD(n_factors = 50, random_state = 0)

# 학습 데이터 세트로 학습 후 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

<h4> 판다스 DataFrame기반에서 동일하게 재 수행

In [17]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv("./ml-latest-small/ratings.csv")

# load_from_df에서는 reader에서 sep 구분 필요 X
reader = Reader(rating_scale = (0.5, 5.0))

# ratings DataFrame 에서 컬럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 함
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)
trainset, testset = train_test_split(data, test_size = .25, random_state = 0)

algo = SVD(n_factors = 50, random_state = 0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

## 교차 검증(Cross Validation)과 하이퍼 파라미터 튜닝
<h4> cross_validate()를 이용한 교차 검증

In [19]:
from surprise.model_selection import cross_validate

# Pandas DataFrame에서 Surprise Dataset으로 데이터 로딩 
ratings = pd.read_csv("./ml-latest-small/ratings.csv")
reader = Reader(rating_scale=(0.5,5.0))
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)

algo = SVD(random_state = 0)
cross_validate(algo, data, measures = ['RMSE','MAE'], cv=5, verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8692  0.8704  0.8745  0.8816  0.8775  0.8746  0.0046  
MAE (testset)     0.6678  0.6689  0.6743  0.6781  0.6738  0.6725  0.0038  
Fit time          4.17    4.31    4.48    4.16    4.40    4.30    0.12    
Test time         0.13    0.20    0.12    0.11    0.21    0.15    0.04    


{'test_rmse': array([0.8691715 , 0.87042738, 0.87454439, 0.88160159, 0.87748318]),
 'test_mae': array([0.66778696, 0.66888184, 0.67425223, 0.67806421, 0.67376465]),
 'fit_time': (4.168371200561523,
  4.307858467102051,
  4.475953102111816,
  4.1644957065582275,
  4.3973963260650635),
 'test_time': (0.12950873374938965,
  0.19769525527954102,
  0.12200355529785156,
  0.10892748832702637,
  0.20531439781188965)}

<h4> GridSearchCV 이용

In [20]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터들을 딕셔너리 형태로 지정. 
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse 로 수행 하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures = ['rmse','mae'], cv = 3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8773476340899479
{'n_epochs': 20, 'n_factors': 50}


## Surprise 를 이용한 개인화 영화 추천 시스템 구축
SVD 학습은 TrainSet 클래스를 이용해야 함

In [21]:
# 아래 코드는 train_test_split( )으로 분리되지 않는 Dataset에 
# fit( )을 호출하여 오류 발생
data = Dataset.load_from_df(ratings[['userId','movieId','rating']].reader)
algo = SVD(n_factors = 50, random_state = 0)
algo.fit(data)

AttributeError: 'DataFrame' object has no attribute 'reader'

<h4> DatasetAutoFolds를 이용한 전체 데이터를 TrainSet클래스 변환

In [22]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep =',', rating_scale=(0.5,5.0))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds(ratings_file = './ml-latest-small/ratings_noh.csv', reader=reader)

#전체데이터를 학습데이터로 생성
trainset = data_folds.build_full_trainset()

<h4> SVD로 학습

In [23]:
algo = SVD(n_epochs = 20, n_factors = 50, random_state = 0)
algo.fit(trainset)

In [25]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('./ml-latest-small/movies.csv')

# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인. 
movieIds = ratings[ratings['userId']==9]['movieId']

if movieIds[movieIds == 42].count() == 0:
    print("user 9 - movieId 42 is not ratings\n")
    
print(movies[movies['movieId']==42])

user 9 - movieId 42 is not ratings

    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [26]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [37]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [57]:
def get_unseen_surprise(ratings, movies, userId) :
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성. 
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print("평점 매긴 영화 수 :",len(seen_movies),
          "\n추천 대상(보지않은) 영화 수 : ",len(unseen_movies),
         "\n전체 영화 수 : ",len(total_movies))
    
    return unseen_movies

In [60]:
unseen_movie= get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수 : 46 
추천 대상(보지않은) 영화 수 :  9696 
전체 영화 수 :  9742


In [67]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n = 10) :
    # 알고리즘 객체의 predict() 메서드를 
    # 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 Predictions 객체를 원소로 가짐.
    # [Prediction(uid='9', iid='1', est=3.69), .... ]
    
    # 이를 est 값으로 정렬하기 위해 sortkey_est 함수 정의
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred) :
        return pred.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key = sortkey_est, reverse = True)
    top_predictions = predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions ]
    top_movie_rating = [ pred.est for pred in top_predictions ]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id,title,rating) for id,title,rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n = 10)
print("\nTop-10 추천 영화 list")

for top_movie in top_movie_preds :
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화 수 : 46 
추천 대상(보지않은) 영화 수 :  9696 
전체 영화 수 :  9742

Top-10 추천 영화 list
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
